In [2]:
import numpy as np
import json
import os

def position_matrix(trial : int, joint: str):
    """ 
    return the position matrix of a joint for a trial.
    precondition : 0 < trial < 126
    """
    data_directory = '../basketball/freethrow/data/P0001/'
    data_files = sorted(os.listdir(data_directory))
    trial_data = f'../basketball/freethrow/data/P0001/{data_files[trial - 1]}'

    with open(trial_data, 'r') as file:
        data = json.load(file)
    
    matrix = np.empty((0, 3))
    for i in range(len(data['tracking'])):
        row = np.array(data['tracking'][i]['data']['player'][joint])
        matrix = np.vstack([matrix, row])
    return matrix

In [3]:
def initial_position_vector(trial: int, joint: str):
    """ 
    returns the initial position vector of a joint for a trial.
    """
    matrix = position_matrix(trial, joint)
    return matrix[0]

In [4]:
def vector_relationship(P0, P1):
    """
    checks if vectors P0 and P1 are parallel, anti-parallel, or neither. 
    """
    if np.allclose(P0, P1):
        return "parallel"
    elif np.allclose(np.cross(P0, P1), [0, 0, 0]):
        return "anti-parallel"
    else:
        return "neither"

In [5]:
import math
from pyquaternion import Quaternion

def quaternion(P0, P1): 
    """
    takes position vectors and returns a quartnion representation of rotation
    from position vector P0 to P1. 
    """
    unit_P0 = P0 / np.linalg.norm(P0)
    unit_P1 = P1 / np.linalg.norm(P1)
    
    relation = vector_relationship(unit_P0, unit_P1)

    if relation == "parallel":
        return Quaternion()

    elif relation == "anti-parallel":
        if np.allclose(unit_P0, [1, 0, 0]):
            axis_of_rotation = [0, 1, 0]
        else:
            cross_product = np.cross(unit_P0, [1, 0, 0])
            axis_of_rotation = cross_product / np.linalg.norm(cross_product)
        return Quaternion(axis=axis_of_rotation, radians=math.pi)

    else:
        cross_product = np.cross(unit_P0, unit_P1)
        magnitude = np.linalg.norm(cross_product)
        axis_of_rotation = cross_product / magnitude

        theta = np.arccos(np.dot(unit_P0, unit_P1))
        return Quaternion(axis = axis_of_rotation, radians=theta)

In [7]:
def sequence_quaternions(position_matrix):
    """
    parameters : 

    position_matrix : a 2D numpy array where each row is the 3D position vector
    for each time stamp. 

    sequence_quaternions computes a sequence of quaternions to represent
    the relative rotation of a joint through time.

    returns a numpy array of quaternions ordered by time stamp. 
    """
    quaternions = []

    for i in range(1, len(position_matrix)):
        P0 = position_matrix[i - 1]
        P1 = position_matrix[i]

        quaternions.append(quaternion(P0, P1))
    return np.array(quaternions)

In [8]:
import plotly.graph_objects as go

def graph_joint_positions(trial: int, joint: str):
    """
    graph the trajectory of a joint.
    """

    data = position_matrix(trial, joint)
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        line=dict(color='red', width=2), 
        name="Trajectory"
    ))

    ball = go.Scatter3d(
        x=[x[0]], y=[y[0]], z=[z[0]],
        mode='markers',
        marker=dict(size=6, color='green', opacity=1), 
        name="Start"
    )
    
    fig.add_trace(ball)

    frames = []
    for i in range(1, len(x)):
        frames.append(go.Frame(
            data=[go.Scatter3d(
                x=x[:i+1], y=y[:i+1], z=z[:i+1], 
                mode='lines',
                line=dict(color='red', width=2)
            ), go.Scatter3d(
                x=[x[i]], y=[y[i]], z=[z[i]], 
                mode='markers',
                marker=dict(size=6, color='blue', opacity=1)
            )],
            name=f"frame_{i}"
        ))

    fig.frames = frames

    fig.update_layout(
        title=f"Joint Trajectory for Trial {trial} - {joint}",
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            aspectmode='cube' 
        ),
        scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)), 
        width=800,
        height=800,
        margin=dict(l=0, r=0, b=0, t=40), 
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]
            )]
        )]
    )
    
    fig.show()


In [ ]:
class UpperKineticChain:
    def __init__(self, trial: int):
        """
        Initialize the UpperKineticChain object for a given trial.
        
        Parameters:
        - trial (int): The trial number to fetch position data.
        """
        self.trial = trial
        self.shoulder = self.position_matrix(trial, 'R_SHOULDER')
        self.elbow = self.position_matrix(trial, 'R_ELBOW')
        self.wrist = self.position_matrix(trial, 'R_WRIST')
        self.vec1 = self.elbow - self.shoulder  
        self.vec2 = self.wrist - self.elbow 
        self.vec1_quaternions = sequence_quaternions(self.vec1)
        self.vec2_quaternions = sequence_quaternions(self.vec2)
        self.between_quaternions = self.compute_quaternions(self.vec1, self.vec2)
    
    def position_matrix(self, trial, joint):
        """ 
        return the position matrix of a joint for a trial.
        precondition : 0 < trial < 126
        """
        data_directory = '../basketball/freethrow/data/P0001/'
        data_files = sorted(os.listdir(data_directory))
        trial_data = f'../basketball/freethrow/data/P0001/{data_files[trial - 1]}'

        with open(trial_data, 'r') as file:
            data = json.load(file)
        
        matrix = np.empty((0, 3))
        for i in range(len(data['tracking'])):
            row = np.array(data['tracking'][i]['data']['player'][joint])
            matrix = np.vstack([matrix, row])
        return matrix

    def sequence_quaternions(self, position_matrix):
        """
        parameters : 

        position_matrix : a 2D numpy array where each row is the 3D position vector
        for each time stamp. 

        sequence_quaternions computes a sequence of quaternions to represent
        the relative rotation of a joint through time.

        returns a numpy array of quaternions ordered by time stamp. 
        """
        quaternions = []

        for i in range(1, len(position_matrix)):
            P0 = position_matrix[i - 1]
            P1 = position_matrix[i]

            quaternions.append(quaternion(P0, P1))
        return np.array(quaternions)
            
    def compute_quaternions(self, vec1, vec2):
        between_quaternions = []
        for i in range(len(vec1)):
            quaternion_rotation = quaternion(vec1[i], vec2[i])
            between_quaternions.append(quaternion_rotation)
        return between_quaternions
